In [ ]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from matplotlib.gridspec import GridSpec
import seaborn as sns

# import data_clean_utils
from scipy.stats import chi2_contingency, f_oneway, jarque_bera, probplot
import plotly.express as px
import timeit

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
import warnings
from sklearn.exceptions import DataConversionWarning


warnings.filterwarnings("default", category=DataConversionWarning)

# import sys
# import importlib
# sys.path.append("../files_")
# import data_clean_script

# importlib.reload(data_clean_script)     # reload

In [30]:
from sklearn import set_config
set_config(transform_output="pandas")

# Import

In [46]:
df=pd.read_csv("../files_/train.csv")

In [47]:
import sys
import importlib
sys.path.append("../files_")
import data_clean_script
importlib.reload(data_clean_script)

<module 'data_clean_script' from '/Users/rabinpoudel/Documents/Code_code_Code/Anaconda/delivery_time_pred/notebooks/../files_/data_clean_script.py'>

In [48]:
df1=data_clean_script.perform_data_cleaning(df)

In [49]:
df.shape,df1.shape

((45593, 20), (45502, 16))

In [50]:
df1.sample(10)

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken,is_weekend,pickup_time_minutes,order_time_of_day,distance,distance_type
20564,NaN,NaN,sandstorms,low,0,meal,motorcycle,1.0,no,NaN,23,0,NaN,NaN,NaN,NaN
16143,36.0,4.6,fog,low,0,drinks,motorcycle,1.0,no,metropolitian,28,0,10.0,morning,3.064361,short
25145,38.0,4.1,fog,low,0,meal,motorcycle,2.0,no,metropolitian,41,0,15.0,night,19.889978,very_long
13475,25.0,4.0,stormy,low,0,buffet,motorcycle,0.0,no,metropolitian,15,0,10.0,morning,1.520369,short
37360,22.0,4.8,sandstorms,medium,0,meal,motorcycle,1.0,no,metropolitian,20,1,10.0,evening,7.683919,medium
19208,29.0,4.8,windy,medium,0,meal,motorcycle,1.0,no,metropolitian,23,0,15.0,evening,4.656945,short
12274,37.0,4.6,fog,medium,2,snack,motorcycle,1.0,no,metropolitian,16,1,15.0,afternoon,6.232242,medium
32959,25.0,4.9,cloudy,low,0,drinks,motorcycle,1.0,no,metropolitian,17,0,10.0,night,7.562818,medium
43478,31.0,5.0,fog,low,1,drinks,scooter,1.0,no,metropolitian,17,0,15.0,night,4.610027,short
33386,30.0,4.7,windy,jam,2,drinks,electric_scooter,1.0,no,metropolitian,25,1,10.0,night,4.528289,short
